In [ ]:
#Base Libraries
import os 
import sys
import json
import csv

#Core Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pygwalker as pyg
import datetime as dt
#Model Libraries
from bertopic import BERTopic
from sklearn.metrics.pairwise import cosine_similarity
from umap import UMAP
import umap.plot
from sentence_transformers import SentenceTransformer

#ML Libraries
import torch 


In [ ]:

#Importing Configs
# Define the path where config.py is located
os.chdir('/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling')
config_file_path = os.getcwd()
print(config_file_path)

# Add this path to the sys.path
sys.path.append(config_file_path)

import config

#Configs
database_file = config.database
database_folder = config.database_folder
bert_models = config.bert_models
bert_models_local = config.bert_models_local

In [ ]:
Body = 'Fed'
Model = 'Beige Book'
Model_Subfolder = f'/{Body} Texts/{Model}'
Model_Folder = config.texts
Model_Folder = Model_Folder + Model_Subfolder
print(Model_Folder)

df = pd.read_csv(f"{Model_Folder}/{Model}_texts.csv")  
docs = df["segment"].to_list()
timestamps = df['date'].to_list()
type = df['type'].to_list()

#BERT Models
topic_model_Beige_Book = torch.load(f"{bert_models_local}/topic_model_Beige Book.pt")
topic_model_all = torch.load(f"{bert_models_local}/topic_model_IMF Stability Reports.pt")

In [ ]:
topics_over_time = topic_model_Beige_Book.topics_over_time(docs, timestamps, nr_bins=100)

df_tot = pd.DataFrame(topics_over_time, columns=['Topic', 'Words', 'Frequency', 'Timestamp'])
df_tot['Timestamp'] = pd.to_datetime(df_tot['Timestamp'], format='%Y-%m-%d').dt.to_period('D')
df_tot.to_csv(f"{bert_models_local}/tot.csv", index=True)

In [ ]:
topics_per_class = topic_model_Beige_Book.topics_per_class(docs, classes = type)
print(topics_per_class)

# Save topic-terms barcharts as HTML file
topic_model_Beige_Book.visualize_barchart(top_n_topics = 100, n_words=8).write_html(f"{bert_models}/barchart.html")

In [ ]:

df_tpc = pd.DataFrame(topics_per_class, columns=['Topic', 'Words', 'Frequency', 'Class'])
df_tpc.to_csv(f"{bert_models_local}/tpc.csv", index=True)

In [ ]:
print(topics_over_time)

In [ ]:
categories_all = df_tpc['Class'].to_list()
categories = list(set(categories_all))
print(categories)
topics = topic_model_all.get_topics()
print(topics)

In [ ]:

pivot_df_tot = df_tot.pivot(index='Timestamp', columns='Topic', values='Frequency')

pivot_df_tot = pivot_df_tot.fillna(0)
print(pivot_df_tot.head())

pivot_df_tot.to_csv(f"{bert_models_local}/pivot_df_tot.csv", index=True)

In [ ]:
gwalker = pyg.walk(df_tot)

In [ ]:
gwalker = pyg.walk(df_tpc)

In [ ]:
mapper = umap.UMAP().fit(pendigits.data)
